In [43]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt

In [44]:
def test_signal(spread, stock, future, name):
    test_data = spread
    spread_upper = spread['0'].quantile(0.8)
    spread_lower = spread['0'].quantile(0.2)
    test_data['signal'] = (test_data['0'] > spread_upper) | (test_data['0'] < spread_lower)
    test_data['direction'] = 1 * (test_data['0'] - test_data['0'].shift() > 0)
    indices = test_data.index[(test_data['signal'].shift() == False) & (test_data['signal'] == True)]
    signal_occur = test_data.loc[indices]
    test_stock = stock
    test_stock['direction'] = 1 * (test_stock['price'] - test_stock['price'].shift() > 0)
    test_future = future
    test_future['direction'] = 1 * (test_future['midQ'] - test_future['midQ'].shift() > 0)
    stock_signal = test_stock.loc[indices]
    future_signal = test_future.loc[indices]
    stock_futures = pd.merge(future_signal, stock_signal, left_index=True, right_index=True, how='inner',suffixes=('_futures', '_stock')).dropna()
    signal_data = pd.merge(stock_futures, signal_occur, left_index=True, right_index=True, how='inner').dropna()
    together_diff = ((signal_data['direction'] == signal_data['direction_stock']) & (signal_data['direction'] != signal_data['direction_futures'])).sum()
    stock_lead_larger = ((signal_data['direction'] == signal_data['direction_stock']) & (signal_data['direction'] == signal_data['direction_futures'])).sum()
    future_lead = ((signal_data['direction'] != signal_data['direction_stock']) & (signal_data['direction'] != signal_data['direction_futures'])).sum()
    total = len(indices)
    together_diff_ratio = together_diff / total
    stock_lead_larger_ratio = stock_lead_larger / total
    future_lead_ratio = future_lead / total
    summary = [together_diff_ratio,stock_lead_larger_ratio,future_lead_ratio]
    return summary
    #print(stock_lead,'\n')
    #print(stock_lead_larger ,'\n')
    #print(future_lead ,'\n')

In [45]:
Path_dir = "/Users/lyq/Documents/Github/IndependentProject/SummerIndependentProject_GP/project_data/Chris_data/"    
name_list = []
for i in os.listdir(Path_dir):
    name_list.append(i)
name_list.remove('.DS_Store')

In [46]:
Path_dir = "/Users/lyq/Documents/Github/IndependentProject/SummerIndependentProject_GP/project_data/Chris_data/"
leading_result = []
for _ in tqdm(name_list):
    spread = pd.read_csv(Path + _ + '/spread' +'.csv.gz',index_col=0, parse_dates=True)
    stock = pd.read_csv(Path+ _ + '/stock' +'.csv.gz',index_col=0, parse_dates=True)
    future = pd.read_csv(Path+ _ + '/future' +'.csv.gz',index_col=0, parse_dates=True)
    result = test_signal(spread, stock, future, _)
    leading_result.append(result)

100%|███████████████████████████████████████████| 10/10 [05:07<00:00, 30.72s/it]


In [49]:
together_diff = [item[0] for item in leading_result]
stock = [item[1] for item in leading_result]
future = [item[2] for item in leading_result]
df = pd.DataFrame({'together':together_diff, 'stock' : stock,'future' : future}, index=name_list)

In [50]:
df

,together,stock,future
HCF-2498,0.508681,0.196457,0.294862
NAF-3105,0.537976,0.298523,0.163501
DBF-2610,0.539997,0.306972,0.153031
EHF-1319,0.485414,0.364445,0.150140
QXF-2615,0.489317,0.289830,0.220853
IIF-3006,0.506710,0.237875,0.255414
QWF-2388,0.489673,0.299127,0.211200
PEF-5425,0.492635,0.296327,0.211038
JBF-3443,0.505897,0.191396,0.302707
IPF-3035,0.491116,0.322249,0.186635
